In [ ]:
#imports
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [ ]:
#training data
df_train=pd.read_csv('/content/drive/My Drive/minor/english_dataset/eng1train.csv', encoding="latin-1")
df_train.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,dhonikeepstheglove watch sports minister kire...,1,NONE,NONE
1,hasoc_en_2,@politico no we should remember very clearly t...,0,HATE,TIN
2,hasoc_en_3,@cricketworldcup guess who would be the winner...,1,NONE,NONE
3,hasoc_en_4,corbyn is too politically intellectual for bor...,1,NONE,NONE
4,hasoc_en_5,all the best to teamindia for another swimming...,1,NONE,NONE


In [ ]:
#training and validation split
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_train['text'], df_train['task_1'],random_state=140) 

In [ ]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder() #label encoder
train_y = encoder.fit_transform(train_y) #encoding label for training data
valid_y = encoder.fit_transform(valid_y) #encoding label for validation data

In [ ]:
#testing data
df_test=pd.read_csv('/content/drive/My Drive/minor/english_dataset/eng1test.csv', encoding="latin-1") 
test_y = encoder.fit_transform(df_test['task_1']) #encoding the label of testing data

In [ ]:
# create a tfidf vectorizer object 
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=25999)
tfidf_vect.fit(df_train['text'])

# transform the training and validation data using count vectorizer object
xtrain_tfidf =  tfidf_vect.transform(train_x)  #get tfidf vector features for training data
xvalid_tfidf =  tfidf_vect.transform(valid_x)  #get tfidf vector features for validation data
x_tfidf =  tfidf_vect.transform(df_test['text'])  #get tfidf vector features for testing data

In [ ]:
def model(classifier, feature_vector_train, label, feature_vector_valid,feature_vector_test):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions_valid = classifier.predict(feature_vector_valid)

    # predict the labels on testing dataset
    predictions_test = classifier.predict(feature_vector_test)

    print("classification report for validation")
    print(metrics.classification_report(predictions_valid,valid_y))
    print("classification report for testing")
    print(metrics.classification_report(predictions_test,test_y))


In [ ]:
# calling the xgboost classifier model for training ,validation and testing
model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc(),x_tfidf.tocsc())

classification report for validation
              precision    recall  f1-score   support

           0       0.19      0.64      0.29       174
           1       0.93      0.63      0.75      1289

    accuracy                           0.63      1463
   macro avg       0.56      0.63      0.52      1463
weighted avg       0.84      0.63      0.70      1463

classification report for testing
              precision    recall  f1-score   support

           0       0.39      0.70      0.50       161
           1       0.94      0.82      0.88       992

    accuracy                           0.81      1153
   macro avg       0.67      0.76      0.69      1153
weighted avg       0.87      0.81      0.83      1153

